In [1]:
import numpy as np

In [2]:
class Variable:
    """ステップ１のVariableクラスを変更"""
    def __init__(self, data):
        if data is not None: # dataがNoneでなく
            if not isinstance(data, np.ndarray): # ndarrayインスタンスでもない場合
                raise TypeError("{} is not supported".formatf(type(data)))
        self.data = data
        self.grad = None
        self.creator = None # 変数視点では関数は生みの親（creator）
    
    def set_creator(self, func):
        """creatorを設定"""
        self.creator = func 

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) # 逆伝播の最初の勾配1を生成
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() # 1.関数を取得
            x, y = f.input, f.output # 2.関数の入出力を取得
            x.grad = f.backward(y.grad)
            
            if x.creator is  not None:
                funcs.append(x.creator) # 4.自分より一つ前の変数のbackwardメソッドをリストに追加

In [3]:
class Function:
    """
    ステップ２のFunctionクラスを変更
    新たに
    ・微分を行う逆伝播の機能
    ・forwardメソッドを呼ぶ際に，入力されたVariableインスタンスを保持する機能
    """
    def __call__(self, input):# f=Function()としたとき，f(...)と書くことでメソッドを呼び出せる
        x = input.data # データを取り出す
        y = self.forward(x) # 実際の計算
        output = Variable(as_array(y)) # ndarrayとして返す
        output.set_creator(self) # 出力変数に生みの親を覚えさせる
        self.input = input # 入力された変数を覚える
        self.output = output # 出力も覚える
        return output
    
    def forward(self, x):
        raise NotImplementedError() # このメソッドは継承して実装すべきことをアピール
    
    def backward(self, gy):
        raise NotImplementedError() 

In [4]:
class Square(Function):
    """backwardメソッドを追加"""
    def forward(self, x):
        return x ** 2
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx
    
# ステップ３　関数の連結

class Exp(Function):
    """backwardメソッドを追加"""
    def forward(self, x):
        return np.exp(x)
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

# 9.1 Pythonの関数として利用

In [11]:
def square(x):
    return Square()(x)
def exp(x):
    return Exp()(x)

# 9.2 backwardメソッドの簡略化

In [12]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.backward()
print(x.grad)

3.297442541400256


In [13]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

# 10.1 Pythonのユニットテスト

In [16]:
import unittest

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)
    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)
    def test_gradient_check(self):
        x = Variable(np.random.rand(1))
        y = square(x)
        y.backward()
        num_grad = numerical_diff(square, x)
        flg = np.allclose(x.grad, num_grad) # a, bが近い値かどうか判定
        self.assertTrue(flg)

# 勾配確認で用いる数値微分

In [15]:
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)